# Creando y generando archivos de entrada para obtener la densidad de carga del estado base

## Importando liberías a utilizar

In [15]:
import os

## Definiendo directorio principal

In [16]:
directorio_principal = '/home/marcocarmonaga/Documents/MoB_2-Study-Properties_Quantum_expresso/notebooks-scripts'
os.chdir(directorio_principal)

## Cambiando a carpeta sobre la que se trabajará (directorio secundario)

In [17]:
scf_path = '../data/processed/calculo-de-interfase-y-fonones'
os.chdir(scf_path)

# Defininiendo valores de ecutwfc y fonones

In [18]:
ecutwfcs = [36.0, 40.0, 44.0, 48.0, 52.0]
fonones = ['01', '02', '03', '04', '05', '06']

# Definiendo función para generar archivos .in

In [19]:
def gen_file_interfase(ecutwfc:str):
   crystal = r'{crystal}'
   file_gen = f''' &CONTROL
    calculation='scf',
    prefix='mob2',
    pseudo_dir='/home/marcocarmonaga/Apps/qe-7.0/pseudo',
    outdir='./tmp',
    wf_collect=.true.,
 /
 &SYSTEM
    ibrav =  4,
    celldm(1) = 5.71334,
    celldm(3) = 1.10154,
    nat =  3,
    ntyp = 2,
    nbnd = 20,
    ecutwfc = {ecutwfc},
    ecutrho = {ecutwfc * 14},
    occupations = 'smearing',
    smearing = 'gauss',
    degauss = 0.015,
 /
 &ELECTRONS
    diagonalization = 'davidson'
    mixing_mode = 'plain',
    conv_thr =  1.0d-8
    mixing_beta = 0.7 
 /
ATOMIC_SPECIES
   Mo  92.906  Mo_ONCV_PBE-1.2.upf
   B   10.81   B_ONCV_PBE-1.2.upf
ATOMIC_POSITIONS {crystal}
   Mo 0.00 0.00 0.00
   B .6666666666666666  .3333333333333333  0.5
   B .3333333333333333  .6666666666666666  0.5
K_POINTS automatic
   18 18 18   0 0 1'''
   return file_gen

In [20]:
def gen_file_fonones(fonon:str):
   file_gen = f'''Phonon calc for qp{fonon}
 &INPUTPH
    prefix='mob2',
    tr2_ph=1.0d-14,
    outdir='./tmp',
    low_directory_check=.true.,
    amass(1)=95.94,
    reduce_io=.true.,
    ldisp=.true.,
!    lqdir=.true.,
    nq1=3,
    nq2=3,
    nq3=3,
    start_q={fonon}, last_q={fonon},
    fildyn="x000.dyn",
 /'''
   return file_gen

In [21]:
for ecutwfc in ecutwfcs:
        name_file = str(ecutwfc).replace('.','_')
        os.mkdir(name_file)
        os.chdir(name_file)
        with open('mob2_interfase.in','w') as file:
                file.writelines(gen_file_interfase(ecutwfc))
        for fonon in fonones:
                with open(f'mob2_fonon_{fonon}.in','w') as file:
                        file.writelines(gen_file_fonones(fonon))
        os.mkdir('tmp')
        os.chdir('../')